In [1]:
!pip install --upgrade pyswarm
!pip install pymc3
!pip install --upgrade pactools

  Created wheel for pyswarm: filename=pyswarm-0.6-cp36-none-any.whl size=4481 sha256=925be16e8bd08de49337b09a546df32ce2a65b4ec026cae7e87f0c0ad7a3714b
  Stored in directory: /root/.cache/pip/wheels/37/c5/f6/b33b9ac00040cb95c1f00af982a4197334a672d6de43f4699f
Successfully built pyswarm
     |████████████████████████████████| 92kB 3.9MB/s 


In [2]:
from sklearn.model_selection import train_test_split
from pyswarm import pso
from os import path
import os
import requests
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import numpy
import sys
import numpy as np
from numpy import loadtxt
from numpy import array
from numpy.random import choice
import pandas as pd
import time
import random
import statistics
import pandas
import math
import csv
import random
import logging
from pymc3 import *
import pymc3 as pm
from functools import reduce
from operator import add
from tqdm import tqdm
import geopy.distance
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from theano import shared
from sklearn import preprocessing
print('Running on PyMC3 v{}'.format(pm.__version__))
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow
from tensorflow.keras import datasets, layers, models
from keras.utils import np_utils
from keras.utils import to_categorical

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.ticker import MaxNLocator

#TESNORFOW
import tensorflow as tf
from tensorflow import keras
from keras import datasets, layers, models

#KERAS LIBRARIES
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout , Flatten,BatchNormalization,Conv2D,MaxPooling2D, Activation,LSTM,Embedding,Input,GlobalAveragePooling2D
from keras.regularizers import l1, l2, l1_l2
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend 
from keras.utils import np_utils
from keras.utils import to_categorical

from numpy import savetxt

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Running on PyMC3 v3.7


In [3]:
def data1():
  train1 = np.load('/content/drive/My Drive/NumpyArrayCovidx/train.npy',allow_pickle=True)
  train_labels1 = np.load('/content/drive/My Drive/NumpyArrayCovidx/train_labels.npy',allow_pickle=True)
  train2,test1, train_labels2,test_labels1 = train_test_split(train1, train_labels1, test_size=0.2,random_state=42)
  x_train=train2/225.0
  y_train = pd.get_dummies(train_labels2)

  x_test=test1/225.0
  y_test = pd.get_dummies(test_labels1)
  return x_train,y_train,x_test,y_test

In [4]:
x_train, y_train, x_test, y_test = data1()

In [5]:
IMG_SHAPE1=(64,64,3) # θα το προσαρμόσουμε

resnet=keras.applications.resnet_v2.ResNet152V2(input_shape=IMG_SHAPE1,
                                               include_top=False,
                                               weights='imagenet')

234553344/234545216 [==============================] - 2s 0us/step


In [6]:
#fine_tuning,lstm_units,dropout,learning_rate
lb=[1,1,0.0,0.001]
ub=[150,10,0.6,0.2]

In [7]:
def create_model_lstm_newresenet(x): 
  print(x[0],x[1],x[2],x[3])
  IMG_SHAPE1=(64,64,3) # θα το προσαρμόσουμε

  resnet=keras.applications.resnet_v2.ResNet152V2(input_shape=IMG_SHAPE1,
                                               include_top=False,
                                               weights='imagenet')

  tempmod=resnet
  for layer in tempmod.layers[:(-1)*int(round(x[0]))]:
    layer.trainable = False
  model = tf.keras.Sequential()
  model.add(tempmod)
  layer_2 =layers.Flatten() 
  model.add(layer_2)

  model.add(layers.Reshape((layer_2.output_shape[1],1)))
  model.add(layers.LSTM(int(round(x[1])),return_sequences=True))
  model.add(layers.Dropout(x[2]))
  model.add(layers.Flatten())
  model.add(keras.layers.Dense(3,activation="softmax"))
  if x[3]< 0.003:
    learning_rate = 0.001
  elif x[3]< 0.0075:
    learning_rate = 0.005
  elif x[3]< 0.015:
    learning_rate = 0.01
  elif x[3]< 0.035:
    learning_rate = 0.02
  elif x[3]< 0.075:
    learning_rate = 0.05
  elif x[3]< 0.125:
    learning_rate = 0.1
  elif x[3]< 0.175:
    learning_rate = 0.15
  else:
    learning_rate = 0.2

  opt = keras.optimizers.Adam(learning_rate=learning_rate)
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
  return model

In [8]:
EarlyStopper = EarlyStopping(patience=4, monitor='val_loss', mode='min')
count = 0

In [9]:
def apple(x):
  model = create_model_lstm_newresenet(x)
  model.fit(x_train, y_train, epochs=20, batch_size=1000, verbose=1,validation_data=(x_test, y_test),callbacks=[EarlyStopper])
  loss, acc = model.evaluate(x_test, y_test, verbose=1)
  if acc>0.9:
    global count 
    count = count+1
    model.save(f"/content/drive/My Drive/saved_models/pso_resnet_lstm/model-{count}-{round(acc, 3)}-{round(loss, 3)}")
    savetxt(f"/content/drive/My Drive/saved_models/pso_resnet_lstm/data-{count}.csv", x, delimiter=',')
  return loss 

In [10]:
xopt, fopt = pso(apple, lb, ub, swarmsize=10, omega=0.5, phip=0.5, phig=1.0, maxiter=30, minstep=1)
print ("Best position"+str(xopt))
print ("Loss:" + str(fopt))

120.29710485230191 1.5573587821248176 0.43613585486086365 0.1201616575439046
Epoch 1/20
17/17 [==============================] - 22s 1s/step - loss: 0.6173 - accuracy: 0.7338 - val_loss: 2.3108 - val_accuracy: 0.4795
Epoch 2/20
17/17 [==============================] - 19s 1s/step - loss: 0.2413 - accuracy: 0.9147 - val_loss: 1.0903 - val_accuracy: 0.6372
Epoch 3/20
17/17 [==============================] - 19s 1s/step - loss: 0.1819 - accuracy: 0.9340 - val_loss: 0.7342 - val_accuracy: 0.7979
Epoch 4/20
17/17 [==============================] - 19s 1s/step - loss: 0.1300 - accuracy: 0.9532 - val_loss: 0.4405 - val_accuracy: 0.8781
Epoch 5/20
17/17 [==============================] - 19s 1s/step - loss: 0.1033 - accuracy: 0.9641 - val_loss: 0.5452 - val_accuracy: 0.8653
Epoch 6/20
17/17 [==============================] - 19s 1s/step - loss: 0.0959 - accuracy: 0.9662 - val_loss: 0.4758 - val_accuracy: 0.8677
Epoch 7/20
17/17 [==============================] - 19s 1s/step - loss: 0.0626 - ac

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-7e2bdf16b76a>", line 1, in <module>
    xopt, fopt = pso(apple, lb, ub, swarmsize=10, omega=0.5, phip=0.5, phig=1.0, maxiter=30, minstep=1)
  File "/usr/local/lib/python3.6/dist-packages/pyswarm/pso.py", line 145, in pso
    fx = obj(x[i, :])
  File "/usr/local/lib/python3.6/dist-packages/pyswarm/pso.py", line 74, in <lambda>
    obj = lambda x: func(x, *args, **kwargs)
  File "<ipython-input-9-5bd49e442848>", line 3, in apple
    model.fit(x_train, y_train, epochs=20, batch_size=1000, verbose=1,validation_data=(x_test, y_test),callbacks=[EarlyStopper])
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py", line 108, in _method_wrapper
    return method(self, *args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/

KeyboardInterrupt: ignored